In [ ]:
!pip install bert-score
!pip install datasets
!pip install sentence-transformers datasets accelerate transformers wandb
!pip install sentence-transformers torch sklearn bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install peft -q

Segment BAAI/bge-m3 model

In [ ]:
# @title
# ============================================================
# SEGMENTED QA → Fine-tune (MORPH SEG ONLY) + Evaluation
# VERSION 4 — LoRA/PEFT APPROACH (fixes persistent OOM on 15 GB GPU)
#
# ROOT CAUSE of OOM: e5-large (560M params) + Adam optimizer states
# (2x weights) + fp32 activations = ~15 GB before any batch data.
#
# SOLUTION: LoRA (Low-Rank Adaptation)
#   - Freeze all 560M base weights (no grad → no optimizer state for them)
#   - Add tiny trainable adapters (~2-4M params) to attention layers
#   - VRAM drops from ~15 GB → ~3-4 GB during training
#   - Quality loss vs full fine-tune: minimal for retrieval tasks
#
# MODEL: BAAI/bge-m3  (E5-large орнына)
#   - BGE-M3 ерекшелігі: query/passage prefix ҚАЖЕТ ЕМЕС
#     (E5-instruct-тен айырмашылығы — бос prefix)
#   - LoRA target modules: "query" / "value" → "query" / "value"
#     (XLM-R негізіндегі BGE-M3-те атаулар бірдей)
#   - encode_query / encode_passage: екеуі де prefix-сіз
#
# Requires: pip install peft
#
# Metrics:
#   Exact@1 | TokenF1@1 | MeanCos@1(QSim)
#   Semantic@1(ans_cos>=0.85) | BERTScoreF1@1 (optional)
# ============================================================

import os
os.environ["WANDB_DISABLED"]              = "true"
os.environ["TOKENIZERS_PARALLELISM"]      = "false"
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"]     = "expandable_segments:True"

import pandas as pd
import numpy as np
import time, re, json, glob, random
import torch
from pathlib import Path
from collections import Counter

# ── PEFT / LoRA ──────────────────────────────────────────────
try:
    from peft import LoraConfig, get_peft_model, TaskType
    HAS_PEFT = True
except ImportError:
    HAS_PEFT = False
    print("⚠️  peft not installed. Run:  pip install peft")
    print("    Falling back to frozen-backbone training (only pooler trained).")

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoModel

# BERTScore — optional
try:
    from bert_score import score as bert_score_fn
    HAS_BERT_SCORE = True
except ImportError:
    bert_score_fn  = None
    HAS_BERT_SCORE = False


# ================================================================
# CONFIG
# ================================================================
DATA_PATH   = "kazakh_segmented_15000.json"

# ✅ ӨЗГЕРІС 1: Модель ауыстырылды
MODEL_NAME  = "BAAI/bge-m3"              # бұрын: intfloat/multilingual-e5-large-instruct

OUTPUT_PATH = "fine-tuned-kz-model-bge-m3"  # бұрын: fine-tuned-kz-model-e5-large

TEST_SIZE    = 0.4
EPOCHS       = 3
WARMUP_STEPS = 100
BATCH_SIZE   = 16
SEM_THR      = 0.85
SEED         = 42
USE_FP16     = torch.cuda.is_available()

LORA_R       = 16
LORA_ALPHA   = 32
LORA_DROPOUT = 0.05
# ✅ ӨЗГЕРІС 2: BGE-M3 XLM-R негізінде — layer атаулары бірдей,
#    бірақ нақты атауларды модель граф арқылы тексереміз (төменде)
LORA_TARGETS = ["query", "value"]


# ================================================================
# REPRODUCIBILITY
# ================================================================
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(SEED)


# ================================================================
# FILE UTILITIES
# ================================================================
def find_data_path(p: str) -> str:
    if Path(p).exists():
        return p
    for c in [f"/content/{p}", f"/content/drive/MyDrive/{p}"]:
        if Path(c).exists():
            return c
    name = Path(p).name
    hits = glob.glob(f"**/{name}", recursive=True)
    if hits:
        return hits[0]
    near = glob.glob("**/*.json", recursive=True)
    raise FileNotFoundError(
        f"File not found: {p}\nPWD: {Path.cwd()}\n"
        f"Nearby .json files:\n" + "\n".join(near[:30])
    )


def _normalize_records(data: list) -> list:
    out = []
    for x in data:
        if not isinstance(x, dict):
            continue
        q = x.get("question") or x.get("instruction") or ""
        a = x.get("answer")   or x.get("response")    or ""
        q, a = str(q).strip(), str(a).strip()
        if q and a:
            out.append({"question": q, "answer": a})
    if not out:
        raise ValueError("No valid question/answer pairs found.")
    return out


def load_qa_records(path: str) -> list:
    text = Path(path).read_text(encoding="utf-8", errors="ignore").strip()
    if not text:
        raise ValueError(f"File is empty: {path}")

    if text[0] == "[":
        try:
            return _normalize_records(json.loads(text))
        except Exception:
            pass

    lines = [ln.strip().rstrip(",") for ln in text.splitlines() if ln.strip()]
    if lines and lines[0].startswith("{"):
        recs, ok = [], True
        for ln in lines:
            try:
                recs.append(json.loads(ln))
            except Exception:
                ok = False; break
        if ok and recs:
            return _normalize_records(recs)

    objs, buf, depth = [], [], 0
    in_str = esc = started = False
    for ch in text:
        if not started:
            if ch == "{":
                started, depth, buf = True, 1, ["{"]
            continue
        buf.append(ch)
        if in_str:
            if esc:          esc = False
            elif ch == "\\": esc = True
            elif ch == '"':  in_str = False
        else:
            if   ch == '"': in_str = True
            elif ch == "{": depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    try:
                        objs.append(json.loads("".join(buf)))
                    except Exception:
                        pass
                    buf, started = [], False

    if not objs:
        raise ValueError(f"Could not parse JSON. Preview:\n{text[:400]}")
    return _normalize_records(objs)


# ================================================================
# TEXT UTILITIES
# ================================================================
def clean_segmented_text(text: str) -> str:
    t = re.sub(r"@@\s*", "", str(text))
    t = re.sub(r"\s+", " ", t).strip()
    t = re.sub(r"\s+([,.!?;:])", r"\1", t)
    t = re.sub(r"\s*([-/])\s*", r"\1", t)
    return t


def normalize_text(t: str) -> str:
    return re.sub(r"\s+", " ", str(t).lower().strip())


KZ_PAT = re.compile(
    r"[a-zA-Zа-яА-ЯәғқңөұүһіӘҒҚҢӨҰҮҺІ0-9]+"
)

def tokens(text: str) -> list:
    return KZ_PAT.findall(normalize_text(text))


def token_f1(pred: str, gold: str) -> float:
    p, g = tokens(pred), tokens(gold)
    if not p and not g: return 1.0
    if not p or  not g: return 0.0
    pc, gc = Counter(p), Counter(g)
    inter  = sum((pc & gc).values())
    if inter == 0: return 0.0
    prec = inter / len(p)
    rec  = inter / len(g)
    return (2 * prec * rec) / (prec + rec + 1e-12)


# ================================================================
# BGE-M3 ENCODING
#
# ✅ ӨЗГЕРІС 3: BGE-M3 — E5-instruct-тен НЕГІЗГІ АЙЫРМАШЫЛЫҚ
#
#   E5-instruct:
#     query   → "Instruct: ...\nQuery: {text}"   (міндетті prefix)
#     passage → тікелей мәтін
#
#   BGE-M3:
#     query   → тікелей мәтін  (prefix ҚАЖЕТ ЕМЕС)
#     passage → тікелей мәтін  (prefix ҚАЖЕТ ЕМЕС)
#
#   BGE-M3 ресми нұсқаулығы (BAAI):
#     "No instruction is needed for either queries or passages."
#
#   encode_query == encode_passage (екеуі де prefix-сіз)
#   Бірақ функцияларды бөліп сақтаймыз — болашақта қажет болса
#   оңай өзгертуге мүмкіндік береді.
# ================================================================

def _nan_guard(vecs: np.ndarray, label: str) -> np.ndarray:
    bad = ~np.isfinite(vecs).all(axis=1)
    if bad.any():
        print(f"  ⚠ {bad.sum()} {label} vectors had NaN/Inf → zeroed.")
        vecs[bad] = 0.0
    return vecs


def encode_query(mdl: SentenceTransformer, texts: list) -> np.ndarray:
    # ✅ BGE-M3: prefix жоқ — мәтінді тікелей береміз
    vecs = mdl.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
        batch_size=32,
        show_progress_bar=False,
    )
    return _nan_guard(vecs, "query")


def encode_passage(mdl: SentenceTransformer, texts: list) -> np.ndarray:
    # ✅ BGE-M3: passage да prefix-сіз
    vecs = mdl.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
        batch_size=32,
        show_progress_bar=False,
    )
    return _nan_guard(vecs, "passage")


# ================================================================
# LOAD DATA
# ================================================================
data_path      = find_data_path(DATA_PATH)
rows           = load_qa_records(data_path)
segmented_data = pd.DataFrame(rows)
print(f"[LOADED] rows={len(segmented_data)} | path={data_path}")

clean_data = segmented_data.copy()
clean_data["question"] = clean_data["question"].apply(clean_segmented_text)
clean_data["answer"]   = clean_data["answer"].apply(clean_segmented_text)


# ================================================================
# TRAIN / TEST SPLIT
# ================================================================
idx = np.arange(len(segmented_data))
train_idx, test_idx = train_test_split(
    idx, test_size=TEST_SIZE, random_state=SEED, shuffle=True
)
seg_train = segmented_data.iloc[train_idx].reset_index(drop=True)
cln_train = clean_data.iloc[train_idx].reset_index(drop=True)
cln_test  = clean_data.iloc[test_idx].reset_index(drop=True)
print(f"[SPLIT] train={len(cln_train)} | test={len(cln_test)}")


# ================================================================
# LOAD BASE MODEL
# ================================================================
print(f"\n[MODEL] Loading {MODEL_NAME} ...")
model = SentenceTransformer(MODEL_NAME)


# ================================================================
# BGE-M3 LoRA TARGET MODULES АВТОМАТТЫ АНЫҚТАУ
#
# ✅ ӨЗГЕРІС 4: BGE-M3 attention layer атауларын автоматты тексеру.
#    Егер "query"/"value" атаулары жоқ болса (кей нұсқаларда
#    "q_proj"/"v_proj" деп аталуы мүмкін), дұрыс атауды табамыз.
# ================================================================
def detect_lora_targets(st_model: SentenceTransformer) -> list:
    """BGE-M3 backbone-нің нақты attention projection атауларын табу."""
    backbone    = st_model[0].auto_model
    linear_names = set()
    for name, module in backbone.named_modules():
        if isinstance(module, torch.nn.Linear):
            # тек соңғы бөлікті аламыз: encoder.layer.0.attention.self.query → query
            leaf = name.split(".")[-1]
            linear_names.add(leaf)

    # Приоритет тәртібімен тексереміз
    candidates = [
        ["query", "value"],       # XLM-R / BERT стандарт
        ["q_proj", "v_proj"],     # LLaMA / Mistral стиль
        ["q", "v"],               # қысқа нұсқа
    ]
    for pair in candidates:
        if all(t in linear_names for t in pair):
            print(f"[LoRA] Detected target modules: {pair}")
            return pair

    # Ештеңе табылмаса — барлық linear layer (қауіпсіз fallback)
    fallback = list(linear_names)[:4]
    print(f"[LoRA] Standard targets not found. Using fallback: {fallback}")
    return fallback


# ================================================================
# APPLY LoRA TO THE BACKBONE
# ================================================================
def apply_lora(st_model: SentenceTransformer) -> SentenceTransformer:
    backbone = st_model[0].auto_model

    # ✅ Автоматты target анықтау
    detected_targets = detect_lora_targets(st_model)

    lora_cfg = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION,
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=detected_targets,   # ✅ hardcode емес, авто
        bias="none",
    )

    peft_backbone = get_peft_model(backbone, lora_cfg)

    total  = sum(p.numel() for p in peft_backbone.parameters())
    train_ = sum(p.numel() for p in peft_backbone.parameters() if p.requires_grad)
    print(f"[LoRA] Total params : {total/1e6:.1f}M")
    print(f"[LoRA] Trainable    : {train_/1e6:.2f}M  "
          f"({100*train_/total:.2f}% of total)")

    st_model[0].auto_model = peft_backbone
    return st_model


def freeze_backbone(st_model: SentenceTransformer) -> SentenceTransformer:
    for name, param in st_model[0].auto_model.named_parameters():
        param.requires_grad = False
    total  = sum(p.numel() for p in st_model.parameters())
    train_ = sum(p.numel() for p in st_model.parameters() if p.requires_grad)
    print(f"[FROZEN] Trainable params: {train_/1e6:.2f}M / {total/1e6:.1f}M total")
    return st_model


if HAS_PEFT:
    model = apply_lora(model)
    print("[LoRA] Adapters applied. Backbone frozen.")
else:
    model = freeze_backbone(model)
    print("[FROZEN] Backbone frozen (install peft for better results).")

if torch.cuda.is_available():
    alloc = torch.cuda.memory_allocated() / 1e9
    total_vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"[GPU] After model prep: {alloc:.2f} GB / {total_vram:.1f} GB allocated")


# ================================================================
# BUILD TRAINING EXAMPLES  (with hard negatives)
# ✅ ӨЗГЕРІС 5: BGE-M3 үшін e5_wrap_query() ЖОҚ —
#    seg_q тікелей InputExample-ге береміз (prefix-сіз)
# ================================================================
print("\n[TRAIN] Building training examples ...")
train_examples  = []
n_train         = len(seg_train)
all_seg_answers = seg_train["answer"].tolist()

for i in range(n_train):
    seg_q = str(seg_train.loc[i, "question"]).strip()
    seg_a = str(seg_train.loc[i, "answer"]).strip()
    if not seg_q or not seg_a:
        continue
    neg_idx = random.choice([k for k in range(n_train) if k != i])
    neg_a   = str(all_seg_answers[neg_idx]).strip()
    # ✅ BGE-M3: query-ға prefix қоспаймыз
    train_examples.append(
        InputExample(texts=[seg_q, seg_a, neg_a])
    )

if len(train_examples) < 2:
    raise ValueError("Not enough training pairs.")

print(f"[TRAIN] {len(train_examples)} examples | batch_size={BATCH_SIZE}")

train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
)

try:
    train_loss = losses.CachedMultipleNegativesRankingLoss(
        model, mini_batch_size=8
    )
    print("[LOSS] CachedMultipleNegativesRankingLoss")
except AttributeError:
    train_loss = losses.MultipleNegativesRankingLoss(model)
    print("[LOSS] MultipleNegativesRankingLoss")


# ================================================================
# CLEAR VRAM BEFORE TRAINING
# ================================================================
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    free = (torch.cuda.get_device_properties(0).total_memory
            - torch.cuda.memory_allocated()) / 1e9
    print(f"\n[GPU] Free VRAM before fit: {free:.2f} GB")


# ================================================================
# FINE-TUNING
# ================================================================
print(f"\n[FINE-TUNE] Epochs={EPOCHS} | warmup={WARMUP_STEPS} | fp16={USE_FP16}")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=WARMUP_STEPS,
    show_progress_bar=True,
    use_amp=USE_FP16,
    optimizer_params={"lr": 2e-4},
)


# ================================================================
# MERGE LoRA WEIGHTS & SAVE
# ================================================================
if HAS_PEFT:
    print("\n[LoRA] Merging adapter weights into base model ...")
    try:
        model[0].auto_model = model[0].auto_model.merge_and_unload()
        print("[LoRA] Merge complete.")
    except Exception as e:
        print(f"[LoRA] Merge failed ({e}) — saving with adapter weights.")

model.save(OUTPUT_PATH)
print(f"[SAVED] {OUTPUT_PATH}")

if torch.cuda.is_available():
    torch.cuda.empty_cache()

model = SentenceTransformer(OUTPUT_PATH)
print("[LOADED] Model reloaded from disk.")


# ================================================================
# BUILD KB INDEX
# ================================================================
train_questions = cln_train["question"].tolist()
train_answers   = cln_train["answer"].tolist()

print(f"\n[KB] Indexing {len(train_questions)} training questions ...")
train_q_embeds = encode_query(model, train_questions)
print(f"[KB] Index shape: {train_q_embeds.shape}")


# ================================================================
# RETRIEVAL
# ================================================================
def ask_question(question: str, threshold: float = 0.0):
    q_clean    = clean_segmented_text(question)
    q_vec      = encode_query(model, [q_clean])
    sims       = q_vec @ train_q_embeds.T
    best_idx   = int(np.argmax(sims[0]))
    best_score = float(sims[0][best_idx])

    if not np.isfinite(best_score):
        return "Кешіріңіз, нақты жауап табылмады.", -1, 0.0
    if best_score < threshold:
        return "Кешіріңіз, нақты жауап табылмады.", -1, best_score
    return train_answers[best_idx], best_idx, best_score


# ================================================================
# EVALUATION (batch-encoded)
# ================================================================
def evaluate_system() -> dict:
    print("\n[EVAL] Running ...")
    t0 = time.time()

    gold_qs = cln_test["question"].tolist()
    gold_as = cln_test["answer"].tolist()
    n_test  = len(gold_qs)

    pred_as, qsims = [], []
    for q in gold_qs:
        pred_a, _, qsim = ask_question(q, threshold=0.0)
        pred_as.append(pred_a)
        qsims.append(qsim)

    print("[EVAL] Batch-encoding for semantic similarity ...")
    pred_vecs   = encode_passage(model, pred_as)
    gold_vecs   = encode_passage(model, gold_as)
    ans_cos_all = np.sum(pred_vecs * gold_vecs, axis=1)

    exacts, tf1s, qcos1s, semhits = [], [], [], []
    for i in range(n_test):
        exacts.append(1.0 if normalize_text(pred_as[i]) == normalize_text(gold_as[i]) else 0.0)
        tf1s.append(token_f1(pred_as[i], gold_as[i]))
        qcos1s.append(float(qsims[i]) if np.isfinite(qsims[i]) else 0.0)
        semhits.append(1.0 if float(ans_cos_all[i]) >= SEM_THR else 0.0)

    results = {
        "Exact@1":                   float(np.mean(exacts)),
        "TokenF1@1":                 float(np.mean(tf1s)),
        "MeanCos@1(QSim)":           float(np.mean(qcos1s)),
        "Semantic@1(ans_cos>=0.85)": float(np.mean(semhits)),
        "BERTScoreF1@1":             None,
    }

    if HAS_BERT_SCORE:
        print("[EVAL] Computing BERTScore ...")
        _, _, F1 = bert_score_fn(pred_as, gold_as, lang="kk", verbose=False)
        results["BERTScoreF1@1"] = float(F1.mean())

    elapsed = time.time() - t0
    print("\n=== Evaluation Results ===")
    print(f"  Exact@1:                    {results['Exact@1']:.6f}")
    print(f"  TokenF1@1:                  {results['TokenF1@1']:.6f}")
    print(f"  MeanCos@1(QSim):            {results['MeanCos@1(QSim)']:.6f}")
    print(f"  Semantic@1(ans_cos>=0.85):  {results['Semantic@1(ans_cos>=0.85)']:.6f}")
    if results["BERTScoreF1@1"] is None:
        print("  BERTScoreF1@1:              (pip install bert-score to enable)")
    else:
        print(f"  BERTScoreF1@1:              {results['BERTScoreF1@1']:.6f}")
    print(f"\n  Elapsed: {elapsed:.2f}s | Test samples: {n_test}")
    return results


# ================================================================
# INTERACTIVE DIALOG
# ================================================================
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("Dialog  |  'eval' → evaluation  |  'exit' → quit")
    print("=" * 60)

    while True:
        user_input = input("\nСұрақ: ").strip()
        if user_input.lower() == "exit":
            print("Stopped. 👋")
            break
        if user_input.lower() == "eval":
            evaluate_system()
            continue
        if not user_input:
            continue
        answer, _, qsim = ask_question(user_input, threshold=0.0)
        print(f"\n  Жауап : {answer}")
        print(f"  QSim  : {qsim:.4f}")

[LOADED] rows=14998 | path=kazakh_segmented_15000.json
[SPLIT] train=8998 | test=6000

[MODEL] Loading BAAI/bge-m3 ...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

[LOADED] Model reloaded from disk.

[KB] Indexing 8998 training questions ...
[KB] Index shape: (8998, 1024)

Dialog  |  'eval' → evaluation  |  'exit' → quit

  Жауап : Python — оқуға жеңіл және интуитивті синтаксиске ие тіл. \n` \n` ` ` ` python \n`сан = 5 \n`if сан > 3: \n` print ( \ ` Үлкен сан \ ` ) \n` ` ` `
  QSim  : 0.8690

[EVAL] Running ...
[EVAL] Batch-encoding for semantic similarity ...
[EVAL] Computing BERTScore ...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-multilingual-cased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



=== Evaluation Results ===
  Exact@1:                    0.015167
  TokenF1@1:                  0.555600
  MeanCos@1(QSim):            0.797252
  Semantic@1(ans_cos>=0.85):  0.160667
  BERTScoreF1@1:              0.851556

  Elapsed: 477.85s | Test samples: 6000

Сұрақ: exit
Stopped. 👋
